# 第一部分：告警聚类   
---

##### 对客户原始告警数据进行算法聚类，并保存聚类后的输出结果，输入以及输出结果一般存放在项目路径下的【dataset/原始告警】路径下 
path:为客户原始告警文件路径 </br>
save_path:保存对客户告警去重后的结果 </br>
save_struc:保存全量的告警内容，以及聚类结果 </br>
save_temp:保存每个模版的首条告警内容，以及对应聚类结果 </br>
sim:根据聚类结果调整聚类相似度阈值

In [1]:
from utils import *
import warnings
warnings.filterwarnings("ignore")

C:\Users\1\miniconda3\envs\_tf24\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### 输入输出路径

In [4]:
# 原始告警文件路径
path = "D:/hj/数据/海通告警数据/海通-模板/dataset/LN/dataset/辽宁农信根据告警等级过滤21883.csv"
# 去重后文件保存路径
save_path = "./dataset/原始告警/nl-去重.csv"
# 聚类结果保存路径 （2个文件）
save_struc = "./dataset/聚类模板/ln-struc.csv"
save_temp = "./dataset/聚类模板/ln-temp.csv"
# 抽取的告警模版保存路径
temp_path = "./dataset/聚类模板/nl_extract_temp.csv"
# 聚类相似度阈值
sim = 0.8

### 1. 加载数据 并 去重

In [5]:
read_and_save(path, save_path, content_name="content")   # content_name: 告警列的名字

原始数据量为: (21883, 16)
安装告警内容去重后数据量为： (21883, 16)


### 2. 聚类

In [6]:
file_name = save_path.split('/')[-1]
start_time = time.time()
structured, templateId = parser(file_name, sim, save_path)
print("Used Time:", time.time() - start_time)
display(templateId)

0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\1\AppData\Local\Temp\jieba.cache


0


Loading model cost 0.459 seconds.
Prefix dict has been built successfully.
10185it [00:06, 1447.63it/s]

10000


20232it [00:13, 1478.39it/s]

20000


21883it [00:15, 1456.18it/s]


聚类后的数据维度大小: (21883, 4) (235, 3)
Used Time: 15.484544038772583


,EventId,EventTemplate,Occurrences
0,0,windows 设备宕机,1
1,1,华为交换机宕机,1
2,2,已经宕机 <\d> 分钟,1
3,3,设备不可达,1
4,4,响应<*>分类下的 【<*>】-【<*>】 满足在 <\d> 个检测窗口中 ， 存在 <\d...,3357
...,...,...,...
230,230,【Critical】【EasyViews】<\d>:<\d>:<\d> 大总账系统 / 应用...,1
231,231,【Critical】【EasyViews】<\d>:<\d>:<\d> 大总账系统 / 应用...,1
232,232,f5 宕机,1
233,233,"专家规则推荐告警内容 ：oracle 数据库不可用 , 场景 ID：<\d> 主告警 ID：...",1


### 3. 保存聚类结果

In [7]:
save_(structured, templateId, save_struc, save_temp)

In [ ]:
# 数据量： 175370
# 模板量： 554
# 耗时：   102s

### 4. 每个模版告警抽取一条告警 生成csv文件

In [8]:
df_struc = pd.read_csv(save_struc)                  # 人工修改好的告警模版保存到原来的 save_struc 位置
df_temp = df_struc.groupby('EventId', group_keys=False).apply(lambda x: x.sample(n=1, random_state=0))
print(df_temp.shape)
df_temp.to_csv(temp_path, index=False)

(235, 4)
